In [ ]:
import librosa
import soundfile as sf
import numpy as np
import IPython.display as ipd
from scipy.signal import butter, lfilter
from speech_recognition import Recognizer, AudioFile
import pyaudio
import wave

# Record audio
def record_audio(filename="recording.wav", duration=5):
    # Set up PyAudio
    p = pyaudio.PyAudio()
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    rate = 44100
    
    # Open stream
    stream = p.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)
    frames = []
    
    print("Recording...")
    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    # Stop stream
    print("Recording complete.")
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # Save the recording
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
    
    # Play the audio
    

# Use the function to record and play audio
record_audio(duration=10)
ipd.Audio("recording.wav")

In [ ]:
# Load audio file
def load_audio(filename):
    audio, sample_rate = librosa.load(filename, sr=None)
    return audio, sample_rate

# Save audio file
def save_audio(filename, audio, sample_rate):
    librosa.output.write_wav(filename, audio, sample_rate)

# Apply a high-pass filter
def high_pass_filter(audio, sample_rate, cutoff=200):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(1, normal_cutoff, btype='high', analog=False)
    filtered_audio = lfilter(b, a, audio)
    return filtered_audio

# Apply a low-pass filter
def apply_lowpass_filter(audio, sample_rate, cutoff=4000):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(1, normal_cutoff, btype='low', analog=False)
    filtered_audio = lfilter(b, a, audio)
    return filtered_audio

# Clean audio and remove noise
def clean_audio(filename, output_filename):
    # Load audio file
    audio, sample_rate = librosa.load(filename, sr=None)
    
    # Apply a high-pass filter to remove low-frequency noise
    audio = librosa.effects.preemphasis(audio)
    
    # Apply a low-pass filter to remove high-frequency noise
    audio = apply_lowpass_filter(audio, sample_rate, cutoff=4000)
    
    # Normalize audio
    audio = librosa.util.normalize(audio)
    
    # Save the cleaned audio using soundfile
    sf.write(output_filename, audio, sample_rate)
    print(f"Cleaned audio saved to: {output_filename}")

# Example usage
# Clean audio and remove noise from 'recording.wav'
# Save the cleaned audio as 'cleaned_recording.wav'
clean_audio("recording.wav", "cleaned_recording.wav")
ipd.Audio("cleaned_recording.wav")

In [ ]:
def recognize_speech_from_audio(filename="cleaned_recording.wav"):
    recognizer = Recognizer()
    
    # Read the audio file
    with AudioFile(filename) as source:
        audio_data = recognizer.record(source)
    
    # Convert speech to text using Google's speech recognition API
    try:
        text = recognizer.recognize_google(audio_data, language="en-US")
        print(f"Recognized text: {text}")
    except Exception as e:
        print(f"Speech recognition error: {e}")

# Use the function to recognize speech from the recorded audio file
recognize_speech_from_audio()
